#Problem Statement: 
Predicting alpha signal using microblogging data

#Problem Description:
Alpha signal is used to make purchase decisions on stocks. This is done by calculating seven stock factors currently. To more accurately predict the Alpha signal, data is gathered from a microblogging site: 'StockTwits'. Brokers and stockers and tweet about the company in this microblogging site and tag the company with 'tickers', that start with a $ sign.

Our goal is to do sentiment Analysis on the tweet data gathered from the microbloggin website and create a new stock factor, and inturn use this stock factor to predict the value of Alpha. Therfore, resulting in a better Aplha signal calculation

#Importing Required python libraries

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import json
import pandas_profiling as pp
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import string
import emoji
import matplotlib.pyplot as plt
from nltk.probability import FreqDist
import wordcloud
from wordcloud import WordCloud
from collections import Counter
import unidecode
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from textblob import TextBlob
from sklearn.model_selection import train_test_split
%matplotlib inline

#Reading the JSON File

In [4]:
with open ('PhD/train_data.json') as f:
    json_data = json.load(f)
    json_data = json_data['records']

#Converting the JSON Data into DataFrames

In [62]:
df_json = pd.DataFrame(json_data)
df_json.head(20)

,sentiment_score,stocktwit_tweet,ticker,timestamp
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00
5,0,$FB as usual......RED \nevery day.,$FB,2018-08-02 11:40:49+00:00
6,2,$AET UnitedHealth Group Goes Ex-Dividend Today...,$AET,2018-09-07 15:01:06+00:00
7,4,$NOC let&#39;s see when we can hit 310,$NOC,2018-09-12 20:48:40+00:00
8,2,@InCyD3R @Ferrabi I bet you’re also the one sa...,$AMZN,2018-10-28 01:19:27+00:00
9,2,$AMD Pastor the Stooge,$AMD,2018-08-17 13:35:00+00:00


#Text Preprocessing and Data cleaning


##Converting tweets to lowercase

In [63]:
df_json["text_clean"] = df_json["stocktwit_tweet"].str.lower()
df_json.head()

,sentiment_score,stocktwit_tweet,ticker,timestamp,text_clean
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00,$amd going up but hesitating however chart is ...
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00,@inforlong @mariagascon despite\nchina trade w...
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00,$avgo wtf?
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00,$ph\n new insider filing on: \n muller klaus p...
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00,$fb if it bounces tommorrow do the right thing...


##Removing Tickers and Mentions from Tweets

In [64]:
df_json["text_clean"] = df_json["text_clean"].replace('\$\w+',' ',regex=True)
df_json["text_clean"] = df_json["text_clean"].replace('\@\w+',' ',regex=True)

In [67]:
df_json.head(20)

,sentiment_score,stocktwit_tweet,ticker,timestamp,text_clean
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00,going up but hesitating however chart is ver...
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00,despite\nchina trade war held very well 👍
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00,wtf?
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00,\n new insider filing on: \n muller klaus pet...
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00,if it bounces tommorrow do the right thing a...
5,0,$FB as usual......RED \nevery day.,$FB,2018-08-02 11:40:49+00:00,as usual......red \nevery day.
6,2,$AET UnitedHealth Group Goes Ex-Dividend Today...,$AET,2018-09-07 15:01:06+00:00,unitedhealth group goes ex-dividend today ht...
7,4,$NOC let&#39;s see when we can hit 310,$NOC,2018-09-12 20:48:40+00:00,let&#39;s see when we can hit 310
8,2,@InCyD3R @Ferrabi I bet you’re also the one sa...,$AMZN,2018-10-28 01:19:27+00:00,i bet you’re also the one saying should ...
9,2,$AMD Pastor the Stooge,$AMD,2018-08-17 13:35:00+00:00,pastor the stooge


##Removing URLs from the Tweets


In [68]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r' ', text)

In [69]:
df_json['text_clean'] = df_json.apply(lambda row: remove_urls(row['text_clean']), axis=1)

In [70]:
df_json['text_clean'].head(20)

0       going up but hesitating however chart is ver...
1           despite\nchina trade war   held very well 👍
2                                                  wtf?
3      \n new insider filing on: \n muller klaus pet...
4       if it bounces tommorrow do the right thing a...
5                       as usual......red  \nevery day.
6           unitedhealth group goes ex-dividend today  
7                     let&#39;s see when we can hit 310
8         i bet you’re also the one saying   should ...
9                                     pastor the stooge
10        called it friday hahahaha suckers sold friday
11      exp:7/20/2018|maxpain:292.5|highputoi:250.0(...
12      day in, day out this puppy is making people ...
13                                                     
14                                                     
15      earnings august 06 bmo. 17-aug-18 straddle i...
16                                          buying more
17      mms took out the sls, yet it still won&#

##Removing html tags from the Tweets

In [71]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r' ', text)

In [72]:
df_json['text_clean'] = df_json.apply(lambda row: remove_html(row['text_clean']), axis=1)

In [73]:
df_json['text_clean'].head(20)

0       going up but hesitating however chart is ver...
1           despite\nchina trade war   held very well 👍
2                                                  wtf?
3      \n new insider filing on: \n muller klaus pet...
4       if it bounces tommorrow do the right thing a...
5                       as usual......red  \nevery day.
6           unitedhealth group goes ex-dividend today  
7                     let&#39;s see when we can hit 310
8         i bet you’re also the one saying   should ...
9                                     pastor the stooge
10        called it friday hahahaha suckers sold friday
11      exp:7/20/2018|maxpain:292.5|highputoi:250.0(...
12      day in, day out this puppy is making people ...
13                                                     
14                                                     
15      earnings august 06 bmo. 17-aug-18 straddle i...
16                                          buying more
17      mms took out the sls, yet it still won&#

### Adding Emoticons

In [74]:
EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

##Converting Emoticons to Text

In [110]:
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(","," ").split()), text)
    return text

In [111]:
df_json['text_clean'] = df_json.apply(lambda row: convert_emoticons(row['text_clean']), axis=1)

In [112]:
df_json['text_clean'].head(20)

0       going up but hesitating however chart is ver...
1           despite\nchina trade war   held very well 👍
2                                                  wtf?
3      \n new insider filing on: \n muller klaus pet...
4       if it bounces tommorrow do the right thing a...
5                       as usual......red  \nevery day.
6           unitedhealth group goes ex-dividend today  
7                     let&#39;s see when we can hit 310
8         i bet you’re also the one saying   should ...
9                                     pastor the stooge
10        called it friday hahahaha suckers sold friday
11      exp:7/20/2018|maxpain:292.5|highputoi:250.0(...
12      day in, day out this puppy is making people ...
13                                                     
14                                                     
15      earnings august 06 bmo. 17-aug-18 straddle i...
16                                          buying more
17      mms took out the sls, yet it still won&#

##Converting Emojis to Text

In [113]:
def convert_emojis(text):
    text = emoji.demojize(text)
    return text

In [114]:
df_json['text_clean'] = df_json.apply(lambda row: convert_emojis(row['text_clean']), axis=1)

In [115]:
df_json['text_clean'].head(20)

0       going up but hesitating however chart is ver...
1         despite\nchina trade war   held very well ...
2                                                  wtf?
3      \n new insider filing on: \n muller klaus pet...
4       if it bounces tommorrow do the right thing a...
5                       as usual......red  \nevery day.
6           unitedhealth group goes ex-dividend today  
7                     let&#39;s see when we can hit 310
8         i bet you’re also the one saying   should ...
9                                     pastor the stooge
10        called it friday hahahaha suckers sold friday
11      exp:7/20/2018|maxpain:292.5|highputoi:250.0(...
12      day in, day out this puppy is making people ...
13                                                     
14                                                     
15      earnings august 06 bmo. 17-aug-18 straddle i...
16                                          buying more
17      mms took out the sls, yet it still won&#

In [118]:
TextBlob(df_json['text_clean'][12]).sentiment.polarity

0.375

In [121]:
TextBlob(df_json['text_clean'][12]).sentiment.polarity

0.525

In [122]:
df_json['text_clean'][12]

'day in day out this puppy is making people rich face with tears of joy man shrugging light skin tone'

In [147]:
from nltk.corpus import sentiwordnet as swn

##Removing Punctutations from Tweet

In [119]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans(PUNCT_TO_REMOVE, ' ' * len(string.punctuation))).replace(' '*4, ' ').replace(' '*3, ' ').replace(' '*2, ' ').strip()

In [120]:
df_json['text_clean'] = df_json.apply(lambda row: remove_punctuation(row['text_clean']), axis=1)

In [123]:
df_json['text_clean'].head(20)

0     going up but hesitating however chart is very ...
1     despite\nchina trade war held very well thumbs up
2                                                   wtf
3     new insider filing on \n muller klaus peter\nt...
4     if it bounces tommorrow do the right thing and...
5                              as usual red \nevery day
6             unitedhealth group goes ex dividend today
7                      let 39 s see when we can hit 310
8        i bet you’re also the one saying should be smh
9                                     pastor the stooge
10        called it friday hahahaha suckers sold friday
11    exp 7 20 2018 maxpain 292 5 highputoi 250 0 78...
12    day in day out this puppy is making people ric...
13                                                     
14                                                     
15    earnings august 06 bmo 17 aug 18 straddle impl...
16                                          buying more
17    mms took out the sls yet it still won 39 t

##Stripping whitespaces from Tweets

In [124]:
def line_strip(text):
    text = text.replace('\n',' ')
    return text

In [125]:
df_json['text_clean'] = df_json.apply(lambda row: line_strip(row['text_clean']), axis=1)

In [126]:
df_json['text_clean'].head(20)

0     going up but hesitating however chart is very ...
1      despite china trade war held very well thumbs up
2                                                   wtf
3     new insider filing on   muller klaus peter tra...
4     if it bounces tommorrow do the right thing and...
5                               as usual red  every day
6             unitedhealth group goes ex dividend today
7                      let 39 s see when we can hit 310
8        i bet you’re also the one saying should be smh
9                                     pastor the stooge
10        called it friday hahahaha suckers sold friday
11    exp 7 20 2018 maxpain 292 5 highputoi 250 0 78...
12    day in day out this puppy is making people ric...
13                                                     
14                                                     
15    earnings august 06 bmo 17 aug 18 straddle impl...
16                                          buying more
17    mms took out the sls yet it still won 39 t

##Removing Numbers from Tweets

In [127]:
def removeNumbers(text):
    text = re.sub(r'\d+', '', text)
    return text

In [128]:
df_json['text_clean'] = df_json.apply(lambda row: removeNumbers(row['text_clean']), axis=1)

In [129]:
df_json['text_clean'].head(15)

0     going up but hesitating however chart is very ...
1      despite china trade war held very well thumbs up
2                                                   wtf
3     new insider filing on   muller klaus peter tra...
4     if it bounces tommorrow do the right thing and...
5                               as usual red  every day
6             unitedhealth group goes ex dividend today
7                           let  s see when we can hit 
8        i bet you’re also the one saying should be smh
9                                     pastor the stooge
10        called it friday hahahaha suckers sold friday
11    exp    maxpain   highputoi   Confusion highcal...
12    day in day out this puppy is making people ric...
13                                                     
14                                                     
Name: text_clean, dtype: object

##Removing Empty rows from Dataset

In [130]:
df_json['text_clean'].size

1039131

In [131]:
df_json = df_json[df_json['text_clean']!='']

In [132]:
df_json['text_clean'].size

1011696

In [133]:
appos = {
"aren’t" : "are not",
"can’t" : "cannot",
"couldn’t" : "could not",
"didn’t" : "did not",
"doesn’t" : "does not",
"don’t" : "do not",
"hadn’t" : "had not",
"hasn’t" : "has not",
"haven’t" : "have not",
"he’d" : "he would",
"he’ll" : "he will",
"he’s" : "he is",
"here’s": "here is",
"i’d" : "I would",
"i’d" : "I had",
"i’ll" : "I will",
"i’m" : "I am",
"isn’t" : "is not",
"it’s" : "it is",
"it’ll":"it will",
"i’ve" : "I have",
"let’s" : "let us",
"mightn’t" : "might not",
"mustn’t" : "must not",
"shan’t" : "shall not",
"she’d" : "she would",
"she’ll" : "she will",
"she’s" : "she is",
"shouldn’t" : "should not",
"that’s" : "that is",
"there’s" : "there is",
"they’d" : "they would",
"they’ll" : "they will",
"they’re" : "they are",
"they’ve" : "they have",
"we’d" : "we would",
"we’re" : "we are",
"weren’t" : "were not",
"we’ve" : "we have",
"what’ll" : "what will",
"what’re" : "what are",
"what’s" : "what is",
"what’ve" : "what have",
"where’s" : "where is",
"who’d" : "who would",
"who’ll" : "who will",
"who’re" : "who are",
"who’s" : "who is",
"who’ve" : "who have",
"won’t" : "will not",
"wouldn’t" : "would not",
"y’all":"you all",
"you’d" : "you would",
"you’ll" : "you will",
"you’re" : "you are",
"you’ve" : "you have",
"’re": " are",
"wasn’t": "was not",
"we’ll":" will",
"didn’t": "did not"
}

In [134]:
def apostrophe_lookup(sentence):
    return ' '.join([appos[word] if word in appos else word for word in sentence.split()])

In [ ]:
ord("’")

In [ ]:
ord("'")

In [135]:
df_json['text_clean'] = df_json.apply(lambda row: apostrophe_lookup(row['text_clean']), axis=1)

In [136]:
df_json['text_clean'].tail(20)

1039110                           what color does alisa like
1039111    when analysts cut price targets and stock drop...
1039112    estimize revenue expectations are higher than ...
1039113    back in the day this would fall on a down day ...
1039114    altria group moConfusion announces earnings ep...
1039115    new insider filing on ravener robert d transac...
1039116                            outperforms of all stocks
1039117                                     russia not happy
1039118                                              crushed
1039119    raise happened recently so not much impact on ...
1039120                             this your entrance point
1039121    inverted hammer rd time is a charm what do you...
1039122    m ago sec current event sConfusion report regu...
1039123    do not even play you all know it is gapping up...
1039124    i wonder why guy adami is always bearish on th...
1039125                prepare yourselves for launch closing
1039127                 

##Removing Stop Words from Tweets

In [137]:
STOPWORDS = (stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in ENGLISH_STOP_WORDS])

In [138]:
df_json['text_clean'] = df_json.apply(lambda row: remove_stopwords(row['text_clean']), axis=1)

In [139]:
def remove_diacritics(text):
    return unidecode.unidecode(text)

In [140]:
df_json['text_clean'] = df_json.apply(lambda row: remove_diacritics(row['text_clean']), axis=1)

In [141]:
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime Anywhere Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It is Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great
G9=Genius
GTFO=Get the Fuck Out
IC=I See
ICQ=I Seek you
ILU= I Love You
IMHO=In My Honest Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple Stupid
LDR=Long Distance Relationship
LMAO=Laugh My Ass Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The Ass
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My Ass Off
SK8=Skate
STATS=Your sex and age
SMH=Shaking My Head
ASL=Age Sex Location
THX=Thank You
TTFN=Ta Ta For Now 
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The Fuck
WTG=Way To Go
WUF=Where Are You From
W8=Wait
"""

In [142]:
chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text).lower()

In [143]:
df_json['text_clean'] = df_json.apply(lambda row: chat_words_conversion(row['text_clean']), axis=1)

In [144]:
df_json.head(20)

,sentiment_score,stocktwit_tweet,ticker,timestamp,text_clean
0,3,$AMD going up but hesitating however chart is ...,$AMD,2018-09-19 18:38:28+00:00,going hesitating chart stable going upward
1,3,@inforlong @MariaGascon Despite\nChina trade w...,$CAT,2018-10-09 03:51:06+00:00,despite china trade war held thumbs
2,2,$AVGO WTF?,$AVGO,2018-07-12 13:35:32+00:00,what the fuck
3,2,$PH\n New Insider Filing On: \n MULLER KLAUS P...,$PH,2018-07-19 03:32:50+00:00,new insider filing muller klaus peter transact...
4,3,$FB if it bounces tommorrow do the right thing...,$FB,2018-08-23 19:07:54+00:00,bounces tommorrow right thing get the fuck out
5,0,$FB as usual......RED \nevery day.,$FB,2018-08-02 11:40:49+00:00,usual red day
6,2,$AET UnitedHealth Group Goes Ex-Dividend Today...,$AET,2018-09-07 15:01:06+00:00,unitedhealth group goes ex dividend today
7,4,$NOC let&#39;s see when we can hit 310,$NOC,2018-09-12 20:48:40+00:00,let s hit
8,2,@InCyD3R @Ferrabi I bet you’re also the one sa...,$AMZN,2018-10-28 01:19:27+00:00,bet saying shaking my head
9,2,$AMD Pastor the Stooge,$AMD,2018-08-17 13:35:00+00:00,pastor stooge


In [145]:
from collections import Counter
cnt = Counter()
for text in df_json["text_clean"].values:
    for word in text.split():
        if len(word)>1:
            cnt[word] += 1
        
cnt.most_common(10)

[('face', 55592),
 ('confusion', 54037),
 ('short', 49587),
 ('buy', 48624),
 ('today', 48467),
 ('just', 48411),
 ('stock', 37052),
 ('earnings', 35784),
 ('calls', 35279),
 ('estimize', 34077)]

In [146]:
n_rare_words = 30
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
RAREWORDS

{'"btd"',
 'aaaamazzzing',
 'artard',
 'bouncingback',
 'brzl',
 'chuggs',
 'dikshit',
 'econoline',
 'fastrack',
 'goodrx',
 'igent',
 'kack',
 'kapertrader',
 'kiu',
 'mwhahahaha',
 'nbas',
 'needmemory',
 'ouija',
 'pecans',
 'psychical',
 'pullish',
 'retrade',
 'revange',
 'shippppppppppp',
 'spxu',
 'strugling',
 'transporting',
 'trumpintergestion',
 'uinta',
 'vox'}

#Finding frequently occuring words in Tweets

In [45]:
def collateText(colName):
    text = " ".join(tweet for tweet in df_json[colName])
    print ("There are {} words in the combination of all review.".format(len(text)))
    return text

In [47]:
allText = collateText('text_clean')
tokenized_text=word_tokenize(allText)
print(len(tokenized_text))

There are 42192393 words in the combination of all review.
6579276


In [48]:
tokenized_text = [token for token in tokenized_text if len(token)>1]

In [49]:
print(len(tokenized_text))

6313988


In [50]:
fdist = FreqDist(tokenized_text)
print(fdist)

<FreqDist with 89101 samples and 6313988 outcomes>


### Top 30 frequently occuring words

In [51]:
fdist.most_common(30)

[('face', 55595),
 ('confusion', 54043),
 ('short', 49615),
 ('today', 49040),
 ('buy', 48726),
 ('just', 48435),
 ('stock', 37083),
 ('earnings', 35793),
 ('calls', 35288),
 ('estimize', 34077),
 ('going', 33443),
 ('like', 33364),
 ('day', 31843),
 ('reporting', 29766),
 ('good', 28872),
 ('market', 28779),
 ('shares', 27207),
 ('analysts', 27003),
 ('new', 26657),
 ('quot', 26035),
 ('laughing', 25689),
 ('money', 25504),
 ('tomorrow', 25343),
 ('trade', 25093),
 ('eps', 25076),
 ('report', 24867),
 ('volume', 24675),
 ('week', 24534),
 ('time', 24212),
 ('revenue', 22274)]

In [ ]:
plt.rcParams['figure.figsize'] = [14, 8]
fdist.plot(30,cumulative=False)
plt.show()

## Visualizations

### Top 5 companies with most tweets with sentiment score as 0

In [ ]:
df = df_json[df_json['sentiment_score']== 0]['ticker'].value_counts().head(5).to_frame()
df.reset_index(level=0,inplace=True)
df.columns=['Ticker','Count']

In [ ]:
ax = sns.barplot(x="Ticker", y="Count", data=df)
ax.set_title("Top 5 tickers with negative tweets")

### Top 5 companies with most tweets with sentiment score as 4

In [ ]:
df = df_json[df_json['sentiment_score']== 4]['ticker'].value_counts().head(5).to_frame()
df.reset_index(level=0,inplace=True)
df.columns=['Ticker','Count']
ax = sns.barplot(x="Ticker", y="Count", data=df)
ax.set_title("Top 5 tickers with Positive tweets")

## Plot of number of tweets sorted by sentiment scores 

In [ ]:
sns.countplot(x="sentiment_score",  data=df_json)

## Word Cloud for Frequently occuring words

In [ ]:
wordcloud = WordCloud(background_color="White", width=3000,height=2000).generate_from_frequencies(fdist)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
def collateScoreText(colName):
    text = " ".join(tweet for tweet in df_json[df_json['sentiment_score']==2]['text_clean'])
    print ("There are {} words in the combination of all review.".format(len(text)))
    return text

In [ ]:
def drawWordCloud(score=0,colName='text_clean'):
  token_text=word_tokenize(collateScoreText(colName))
  freqdist = FreqDist(token_text)
  wordcloud = WordCloud(background_color="White", width=3000,height=2000).generate_from_frequencies(freqdist)
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis("off")
  plt.show()

## Word Cloud for most frequent words during very negative tweets

In [ ]:
drawWordCloud(score=0,colName='text_clean')

## Word Cloud for most frequent words during negative tweets

In [ ]:
drawWordCloud(score=1,colName='text_clean')

## Word Cloud for most frequent words during ok tweets

In [ ]:
drawWordCloud(score=2,colName='text_clean')

## Word Cloud for most frequent words during positive tweets

In [ ]:
drawWordCloud(score=3,colName='text_clean')

## Word Cloud for most frequent words during very positive tweets

In [ ]:
drawWordCloud(score=4,colName='text_clean')

## Lemmitization of the Tweets

In [ ]:
def word_token(text):
    return word_tokenize(text)

In [ ]:
word_token(df_json['text_clean'][0])

In [ ]:
df_json['text_tokenize'] = df_json.apply(lambda row: word_token(row['text_clean']), axis=1)

In [ ]:
df_json.head()

In [ ]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [ ]:
df_json['text_clean'] = df_json.apply(lambda row: lemmatize_words(row['text_clean']), axis=1)

In [ ]:
df_json['text_clean'].head(20)

In [ ]:
def pos_tagging(text):
    return TextBlob(text).tags

In [ ]:
df_json['pos_tag'] = df_json.apply(lambda row: pos_tagging(row['text_clean']), axis=1)

In [ ]:
df_json['pos_tag'].head(10)

In [ ]:
allText = collateText('text_clean')
tokenized_text=word_tokenize(allText)
print(len(tokenized_text))
tokenized_text = [token for token in tokenized_text if len(token)>1]
fdist = FreqDist(tokenized_text)
print(fdist)
fdist.most_common(30)

In [ ]:
44743155 - 43967701 

In [ ]:
df_json_model = df_json.copy()

In [ ]:
df_json_model.head()

In [ ]:
drop_cols=['stocktwit_tweet','ticker','timestamp','text_tokenize','pos_tag']
df_json_model = df_json_model.drop(drop_cols,axis=True)
df_json_model.head(10)

In [ ]:
trainX,valX,trainY,valY = train_test_split(df_json.loc[:,df_json.columns!="sentiment_score"],df_json.loc[:,df_json.columns=="sentiment_score"],test_size=0.2,random_state=1)

In [ ]:
print(trainX.shape)
print(valX.shape)
print(trainY.shape)
print(valY.shape)

In [ ]:
valX

In [ ]:
tfidf_transformer = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS,lowercase=True ,norm=None)
trainX_tfidf = tfidf_transformer.fit_transform(trainX['text_clean'])
trainX_tfidf.shape

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
clf = MultinomialNB().fit(trainX_tfidf,trainY)
pred = clf.predict(trainX_tfidf)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(trainY,pred))